## Imports

In [ ]:
import lfd
import pandas as pd
import numpy as np

In [ ]:
DATASET = 'Houses'
SET_ASIDE = []
MODE = 'clustering'

## Data

In [ ]:
data = lfd.Data(f'../../learnfromdata_new/datasets/tabular/{DATASET.lower()}')
data.set_dtypes()
data.df.head(3)

In [ ]:
data.df.describe()

In [ ]:
params = dict(
    set_aside = SET_ASIDE,
    data = dict(
        add_noise = dict(seed=0),
        test_split = dict(test_size=0.2, stratify_col=None, seed=0),
        valid_split = dict(test_size=0.2, stratify_col=None, seed=0),
    ),
    transform = dict(
        uniselector = dict(min_occ=0.01, max_occ=0.99),
        imputer = dict(default_cat='MISSING', default_cont='median'),
        encoder = dict(min_occ=0.05, method='target', target='SalePrice'),
        biselector = dict(threshold=0.8),
        standardizer = dict(),
    ),
    model = dict(
        mode=MODE, seed_train=0,
        base0 = dict(algorithm='gaussianmixture', name='GM', hyper_params=dict(
            n_components=5
        )),
    ),
)

In [ ]:
pipe = lfd.Pipeline(name=f'{DATASET}_clustering').learn(
    params, data=data.copy(), evaluate=False, explain=False)

## Evaluation

In [ ]:
model = pipe.models['GM']

In [ ]:
model.predictions.df

In [ ]:
model.clf.aic(pipe.data.df.loc['Test'])

In [ ]:
plotter = lfd.PlotterModel(theme='dark')

In [ ]:
plotter.scatter(
    pipe.data.df.loc['Test', 'SalePrice'], 
    pipe.data.df.loc['Test', 'LotArea'], 
    model.predictions.df.loc['Test', 'predictions'], 
)

In [ ]:
plotter.histogram(
    model.predictions.select('Train').df.predictions, 
    model.predictions.select('Test').df.predictions, 
    bins=100, normalize=True
)

In [ ]:
model.feature_imp.sort_values('cluster_all', ascending=False)

In [ ]:
pipe.data.df.groupby(model.predictions.df.predictions).median().std().sort_values()